In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16565,2024-03-10,Brasil Nbb,16:00,Brasília,Botafogo,1.98,1.75,162.5,1.87,1.83,1.5,1.85,1.61,OG87sGjh,0.505051,0.571429,0.534759,0.546448,0.076479,0.4,0.4,NaN,NaN,366.720,197.473809,0.538487,365.184,113.115505,0.309749,341.462,329.616,625.30,349.80,1.0,0.0,0.0,0.0,0.087204,0.015289,0.098096,-5.00,-1.000,-0.980000,0.000000,0.0,0.000000,-5.00,-1.000,-0.750000,0.228358,0.2,-0.028358
16566,2024-03-10,Espanha Acb,13:00,Granada,Bilbao,1.61,2.33,162.5,1.87,1.91,-3.5,1.95,2.51,bufHyk5k,0.621118,0.429185,0.534759,0.523560,0.050303,0.6,0.6,NaN,NaN,209.760,109.188714,0.520541,341.052,306.751881,0.899428,166.998,361.434,165.54,291.08,0.0,0.0,0.0,0.0,0.258435,0.014965,0.177569,-0.10,-0.020,-30.500000,0.000000,0.0,0.000000,-5.00,-1.000,-1.330000,0.000000,0.0,0.000000
16567,2024-03-10,Espanha Acb,13:00,Palencia,MoraBanc Andorra,2.06,1.77,166.5,1.86,1.92,1.5,1.90,1.63,t4qMz9Ke,0.485437,0.564972,0.537634,0.520833,0.050409,0.4,0.8,NaN,NaN,433.480,370.134787,0.853868,256.228,183.201026,0.714992,227.296,443.194,237.80,528.90,0.0,1.0,0.0,0.0,0.107081,0.022448,0.108169,0.04,0.008,132.500000,0.000000,0.0,0.000000,-5.00,-1.000,-0.770000,0.000000,0.0,0.000000
16568,2024-03-10,Espanha Acb,14:30,Baskonia,Barcelona,2.12,1.73,168.5,1.81,1.95,1.5,1.97,1.57,IeqeczdI,0.471698,0.578035,0.552486,0.512821,0.049733,0.6,0.4,NaN,NaN,138.406,37.270523,0.269284,115.706,14.339016,0.123926,123.022,124.106,100.62,120.93,0.0,0.0,0.0,0.0,0.143258,0.052657,0.159798,-1.46,-0.292,-3.835616,0.735734,0.6,-0.135734,-0.71,-0.142,-5.140845,0.000000,0.0,0.000000
16569,2024-03-10,Eua Nba,00:00,Portland Trail Blazers,Toronto Raptors,2.20,1.72,226.5,1.80,1.95,1.5,2.00,1.56,vqk9fDzo,0.454545,0.581395,0.555556,0.512821,0.035941,0.8,0.8,NaN,NaN,334.124,76.103051,0.227769,226.174,60.419707,0.267138,320.396,366.484,312.44,172.20,0.0,0.0,0.0,0.0,0.173169,0.056569,0.174790,-5.00,-1.000,-1.200000,0.307256,0.4,0.092744,-3.60,-0.720,-1.000000,0.367414,0.3,-0.067414
16570,2024-03-10,Eua Nba,16:00,Los Angeles Clippers,Milwaukee Bucks,1.75,1.92,221.5,1.83,1.83,-1.5,2.04,1.95,AgbBYnLf,0.571429,0.520833,0.546448,0.546448,0.092262,0.2,0.2,NaN,NaN,193.058,42.967195,0.222561,186.284,17.006097,0.091291,153.272,205.668,182.56,197.64,0.0,0.0,0.0,0.0,0.065509,0.000000,0.031900,-2.65,-0.530,-1.415094,0.742871,0.7,-0.042871,0.46,0.092,10.000000,0.609330,0.4,-0.209330
16571,2024-03-10,Eua Nba,19:00,Orlando Magic,Indiana Pacers,1.83,1.83,227.5,1.83,1.98,-2.5,2.00,2.09,bFv6fUsE,0.546448,0.546448,0.546448,0.505051,0.092896,0.8,0.4,NaN,NaN,168.664,23.811352,0.141176,210.814,59.537022,0.282415,189.482,226.868,199.42,273.36,0.0,0.0,0.0,0.0,0.000000,0.055678,0.031120,1.42,0.284,2.922535,0.533719,0.7,0.166281,-2.22,-0.444,-1.869369,0.487051,0.3,-0.187051
16572,2024-03-10,Eua Nba,20:00,Oklahoma City Thunder,Memphis Grizzlies,1.07,6.34,218.5,1.82,1.98,-4.5,1.99,2.19,8lUulSKs,0.934579,0.157729,0.549451,0.505051,0.092308,0.6,0.6,NaN,NaN,203.172,18.811144,0.092587,260.570,203.350364,0.780406,181.012,489.312,229.62,172.78,0.0,0.0,0.0,0.0,1.005790,0.059546,0.067666,2.32,0.464,0.150862,0.724716,0.9,0.175284,-0.24,-0.048,-111.250000,0.290837,0.4,0.109163
16573,2024-03-10,Itália Liga A,14:15,Virtus Bologna,Milano,1.63,2.29,156.5,1.87,1.91,-2.5,1.83,2.64,nmTZF2lr,0.613497,0.436681,0.534759,0.523560,0.050178,0.6,0.2,NaN,NaN,114.930,9.697917,0.084381,94.518,13.240756,0.140087,108.820,104.924,102.46,100.62,0.0,0.0,0.0,0.0,0.238107,0.014965,0.256267,-0.43,-0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,13:00,Espanha Acb,Granada,Bilbao,162.5,1.87,1.0000,Over
1,16:00,Eua Nba,Los Angeles Clippers,Milwaukee Bucks,221.5,1.83,0.9998,Over
2,19:00,Eua Nba,Orlando Magic,Indiana Pacers,227.5,1.83,1.0000,Over
3,20:00,Eua Nba,Oklahoma City Thunder,Memphis Grizzlies,218.5,1.82,1.0000,Over
4,14:15,Itália Liga A,Virtus Bologna,Milano,156.5,1.87,1.0000,Over
5,13:00,Alemanha Bbl,Braunschweig,Hamburg,171.5,1.80,1.0000,Over
6,13:00,Áustria Liga Zweite,Mattersburg,Kufstein,147.5,1.86,1.0000,Over
7,14:00,Espanha Leb Plata,Alginet,CB Santfeliuenc,147.5,1.85,1.0000,Over
8,00:00,Eua Ncaa,UC Santa Barbara,UC Riverside,146.5,1.91,1.0000,Over
9,01:59,Eua Ncaa,Hawaii,Cal St. Bakersfield,133.5,1.91,1.0000,Over
